In [20]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [21]:
df = pd.read_csv('final_file.csv',encoding="GBK").drop(['Unnamed: 0'], axis=1)
X = df.values[:, 0:216]
Y = df.values[:,216]

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100)

In [3]:
df.head()

,shop_id,date.day_1,date.day_2,date.day_3,date.day_4,date.day_5,date.day_6,date.day_7,date.day_8,date.day_9,...,cate_2_name_快餐,cate_2_name_本地购物,cate_2_name_汤/粥/煲/砂锅/炖菜,cate_2_name_火锅,cate_2_name_烘焙糕点,cate_2_name_烧烤,cate_2_name_网吧网咖,cate_2_name_药店,cate_2_name_超市,count
0,544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,221
1,810,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125
2,163,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,91
3,1977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,137
4,746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68


In [4]:
from sklearn import datasets, linear_model, cross_validation, grid_search
from sklearn.metrics import r2_score 
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

/opt/anaconda/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [5]:
import sklearn
sklearn.__version__

'0.18.1'

In [7]:
r2_results = []
mse_results= []

kf_total = KFold(n_splits=2, shuffle=True, random_state=100)
for train_index, test_index in kf_total.split(X_train):
    x_tr, x_cv = X_train[train_index], X_train[test_index]
    y_tr, y_cv = y_train[train_index], y_train[test_index]
    T_train_xgb = xgb.DMatrix(x_tr, y_tr)
    params = {"objective": "reg:linear", "booster":"gbtree","max_depth":60}
    gbm = xgb.train(dtrain=T_train_xgb,params=params)
    y_pr = gbm.predict(xgb.DMatrix(x_cv))
    r2_results.append(r2_score(y_cv, y_pr))
    mse_results.append(mean_squared_error(y_cv, y_pr))
    
print np.average(r2_results)
print np.average(mse_results)

0.851709436989
2631.5385578


In [10]:
y_pred = gbm.predict(xgb.DMatrix(X_test))
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.987571751453
225.28010213


In [ ]:
params2 = {"objective": "reg:linear", "booster":"gbtree","max_depth":60}
gbm2 = xgb.train(dtrain=T_train_xgb,params=params2)
y_pred2 = gbm2.predict(xgb.DMatrix(X_test))
print(r2_score(y_test, y_pred2))
print(mean_squared_error(y_test, y_pred2))

In [22]:
df1 = df[df['shop_id'] != 1629]
df1 = df[df['shop_id'] != 58]
df1 = df[df['shop_id'] != 1821]
#df['shop_id'] != 58 and df['shop_id'] != 1821]

In [23]:
X = df1.values[:, 0:216]
Y = df1.values[:,216]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100)

In [24]:
r2_results = []
mse_results= []
kf_total = KFold(n_splits=20, shuffle=True, random_state=100)
for train_index, test_index in kf_total.split(X_train):
    x_tr, x_cv = X_train[train_index], X_train[test_index]
    y_tr, y_cv = y_train[train_index], y_train[test_index]
    T_train_xgb = xgb.DMatrix(x_tr, y_tr)
    params_1 = {"booster":"gbtree", "objective": "reg:linear", "eta":0.1, "max_depth":40,"eval_metric":"rmse", 'lambda': 5}
    gbm_p_1 = xgb.train(dtrain=T_train_xgb,params=params_1)
    y_pr = gbm_p_1.predict(xgb.DMatrix(x_cv))
    r2_results.append(r2_score(y_cv, y_pr))
    mse_results.append(mean_squared_error(y_cv, y_pr))
print 'average_r2', np.average(r2_results)
print 'average_mse', np.average(mse_results)
print 'median_r2', np.median(r2_results)
print 'median_mse', np.median(mse_results)

average_r2 0.62050130956
average_mse 6583.3831421
median_r2 0.619683662447
median_mse 6666.51649687


In [28]:
y_pred = gbm_p_1.predict(xgb.DMatrix(X_test))
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.643371327907
6097.39510445


In [29]:
r2_results = []
mse_results= []
kf_total = KFold(n_splits=20, shuffle=True, random_state=100)
for train_index, test_index in kf_total.split(X_train):
    x_tr, x_cv = X_train[train_index], X_train[test_index]
    y_tr, y_cv = y_train[train_index], y_train[test_index]
    T_train_xgb = xgb.DMatrix(x_tr, y_tr)
    params_2 = {"booster":"gbtree", "objective": "reg:linear", "eta":0.1, "max_depth":20,"eval_metric":"rmse", 'lambda': 1.5}
    gbm_p_2 = xgb.train(dtrain=T_train_xgb,params=params_2)
    y_pr = gbm_p_2.predict(xgb.DMatrix(x_cv))
    r2_results.append(r2_score(y_cv, y_pr))
    mse_results.append(mean_squared_error(y_cv, y_pr))
print 'average_r2', np.average(r2_results)
print 'average_mse', np.average(mse_results)
print 'median_r2', np.median(r2_results)
print 'median_mse', np.median(mse_results)

average_r2 0.622400401729
average_mse 6548.40322707
median_r2 0.623494533462
median_mse 6584.91467408


In [31]:
y_pred = gbm_p_2.predict(xgb.DMatrix(X_test))
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.64548838622
6061.19907759


In [35]:
r2_results = []
mse_results= []
kf_total = KFold(n_splits=20, shuffle=True, random_state=100)
for train_index, test_index in kf_total.split(X_train):
    x_tr, x_cv = X_train[train_index], X_train[test_index]
    y_tr, y_cv = y_train[train_index], y_train[test_index]
    T_train_xgb = xgb.DMatrix(x_tr, y_tr)
    params_3 = {"booster":"gbtree", "objective": "reg:linear", "max_depth":60,"eval_metric":"rmse"}
    gbm_p_3 = xgb.train(dtrain=T_train_xgb,params=params_3)
    y_pr = gbm_p_3.predict(xgb.DMatrix(x_cv))
    r2_results.append(r2_score(y_cv, y_pr))
    mse_results.append(mean_squared_error(y_cv, y_pr))
print 'average_r2', np.average(r2_results)
print 'average_mse', np.average(mse_results)
print 'median_r2', np.median(r2_results)
print 'median_mse', np.median(mse_results)

average_r2 0.853752707929
average_mse 2547.76670546
median_r2 0.851616592089
median_mse 2493.81722666


In [36]:
y_pred = gbm_p_3.predict(xgb.DMatrix(X_test))
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.851969030617
2530.93309274


--------

In [37]:
df_1629 = df[df['shop_id'] == 1629]
df_58 = df[df['shop_id'] == 58]
df_1821 = df[df['shop_id'] == 1821]

In [38]:
X_1629 = df_1629.values[:, 0:216]
Y_1629 = df_1629.values[:,216]

X_train_1629, X_test_1629, y_train_1629, y_test_1629 = train_test_split(X_1629, Y_1629, test_size = 0.3, random_state = 100)

X_1821 = df_1821.values[:, 0:216]
Y_1821 = df_1821.values[:,216]

X_train_1821, X_test_1821, y_train_1821, y_test_1821 = train_test_split(X_1821, Y_1821, test_size = 0.3, random_state = 100)

X_58 = df_58.values[:, 0:216]
Y_58 = df_58.values[:,216]

X_train_58, X_test_58, y_train_58, y_test_58 = train_test_split(X_58, Y_58, test_size = 0.3, random_state = 100)

In [84]:
l = [1, 2, 3, 4, 5, 7, 9, 10]
e = [0.1, 0.3, 0.5]
d = [10, 20, 30, 40, 50, 60, 100]

for i in e:
    r2_results = []
    mse_results= []
    kf_total = KFold(n_splits=20, shuffle=True, random_state=100)
    for train_index, test_index in kf_total.split(X_train_58):
        x_tr_58, x_cv_58 = X_train_58[train_index], X_train_58[test_index]
        y_tr_58, y_cv_58 = y_train_58[train_index], y_train_58[test_index]
        T_train_xgb = xgb.DMatrix(x_tr_58, y_tr_58)
        params58_3 = {"booster":"gbtree", "objective": "reg:linear", 'max_depth':20,"eval_metric":"rmse", "eta":0.3, 'lambda':4}
        gbm58_p_3 = xgb.train(dtrain=T_train_xgb,params=params58_3)
        y_pr = gbm58_p_3.predict(xgb.DMatrix(x_cv_58))
        r2_results.append(r2_score(y_cv_58, y_pr))
        mse_results.append(mean_squared_error(y_cv_58, y_pr))
    print 'average_r2', np.average(r2_results)
    print 'average_mse', np.average(mse_results)
    print 'median_r2', np.median(r2_results)
    print 'median_mse', np.median(mse_results)
    print "\n"

average_r2 -1.03205257242
average_mse 338840.081131
median_r2 -0.956453193745
median_mse 329109.719358


average_r2 0.725254295506
average_mse 47241.7385058
median_r2 0.771807815201
median_mse 31640.9026126


average_r2 0.614243754972
average_mse 58428.3394604
median_r2 0.732862127377
median_mse 45311.4213972




In [86]:
r2_results = []
mse_results= []
kf_total = KFold(n_splits=20, shuffle=True, random_state=100)
for train_index, test_index in kf_total.split(X_train_58):
    x_tr_58, x_cv_58 = X_train_58[train_index], X_train_58[test_index]
    y_tr_58, y_cv_58 = y_train_58[train_index], y_train_58[test_index]
    T_train_xgb = xgb.DMatrix(x_tr_58, y_tr_58)
    params58_3 = {"booster":"gbtree", "objective": "reg:linear", 'max_depth':20,"eval_metric":"rmse", "eta":0.3, 'lambda':4}
    gbm58_p_3 = xgb.train(dtrain=T_train_xgb,params=params58_3)
    y_pr = gbm58_p_3.predict(xgb.DMatrix(x_cv_58))
    r2_results.append(r2_score(y_cv_58, y_pr))
    mse_results.append(mean_squared_error(y_cv_58, y_pr))
print 'average_r2', np.average(r2_results)
print 'average_mse', np.average(mse_results)
print 'median_r2', np.median(r2_results)
print 'median_mse', np.median(mse_results)
print "\n"

average_r2 0.725254295506
average_mse 47241.7385058
median_r2 0.771807815201
median_mse 31640.9026126




In [87]:
y_pred = gbm58_p_3.predict(xgb.DMatrix(X_test_58))
print(r2_score(y_test_58, y_pred))
print(mean_squared_error(y_test_58, y_pred))

0.806873469607
25887.2294966
